
### Matrix Multiplication as a Linear Map

Multiplication of $n \times n$ matrices is a bilinear map $M_n \times M_n \to M_n$
and hence there exists a linear map $m : M_n \otimes M_n \to M_n$ such that
$m(a \otimes b) = ab$ for all $a, b \in M_n$. We can generalize matrix multiplication
as a map

$$m_r : M_n \otimes M_n \to M_n : a \otimes b \mapsto arb$$

for some invertible $r \in M_n$, noting that this
specializes to ordinary multiplication when $r$ is the $n \times n$ identity matrix,
$I_n$. We also note that this multiplication has unit $e_r : 1 \mapsto r^{-1}$.

Taking $\otimes$ as the outer product, we can compute the matrix $M_r$ representing
$m_r$ as follows.


In [1]:

# These are just some helper algorithms:

def decl(v, n):
    """
    Declare symbolic variables
    """
    return var(','.join([v + str(i) + str(j)
        for i in range(1, n + 1)
        for j in range(1, n + 1)
    ])) if n > 1 else tuple([var(v + str(1) + str(1))])

def kd(i, j):
    """
    Kronecker delta
    """
    return 1 if i == j else 0

def I(n):
    """
    n x n identity matrix
    """
    return matrix([
        [kd(i, j) for i in range(1, n + 1)]
            for j in range(1, n + 1)
    ])

def m2v(M):
    """
    Serialize M to a vector row-by-row
    """
    m, n = M.dimensions()
    return vector(
        [ M[i][j] for i in range(m) for j in range(n) ]
    )

def v2m(v, m, n):
    """
    Deserialize a vector v to an m x n matrix
    """
    if (len(v) != m * n):
        return None
    else:
        return matrix([
                [ v[j] for j in range(i, i + n) ]
                for i in range(0, m * n, n)
            ])

def matapply(f, P):
    """
    Apply f to all elements of a matrix P
    """
    return matrix([
        [f(x) for x in row]
            for row in P
    ])

def matsimpl(P):
    """
    Simplify each entry of the matrix P
    """
    return matapply(lambda x: x.expand().simplify_rational(), P)

def mathat(Q):
    """
    Left multiplicand of outer product giving matrix for
    (A, B) |---> AQB
    """
    Q = []
    for i in range(N):
        prefix = [0] * (N^2 * i)
        suffix = [0] * (N^3 - N^2 - i*N^2)
        Qi = vector(prefix + [R[i][j] for j in range(N) for i in range(N)] + suffix)
        Q.append(Qi)
    return matrix(Q)

def norm(Q):
    """
    Sum of the squares of all entries in Q
    """
    return sum(sum(x * conjugate(x) for x in row) for row in Q)

def matinv(Q):
    """
    Compute the inverse of Q using Steve's God algorithm
    """
    f = Q.charpoly()
    cs = f.coefficients()
    c = cs[0]
    return (-1/c) * matsimpl(sum([ cs[i + 1] * Q^(i) for i in range(len(cs[1:])) ]))


In [2]:

for N in range(1, 6):

    a = decl('a', N)
    b = decl('b', N)
    r = decl('r', N)

    A = v2m(a, N, N)
    B = v2m(b, N, N)
    R = v2m(r, N, N)

    ARB = A * R * B
#     show(matsimpl(ACB))

    AoB = A.tensor_product(B)
#     show(AoB)

    M = mathat(R).tensor_product(I(N))
    assert(matsimpl(v2m(M * m2v(AoB), N, N)) == matsimpl(ARB))

print('Done')


Done



Given an $n \times n$ matrix $r$, let $v(r)$ be the vector obtained by taking the entries
of $r$ in lexicographic order of the indices and $0_n$ be the row vector consisting of $n$
$0$'s. We then denote:

$$
\hat{r} = \begin{bmatrix}
    v(r^T) & 0_n & \dots & 0_n\\
         0_n & v(r^T) & \dots & 0_n\\
    \vdots & \vdots & \ddots & \vdots\\
         0_n & 0_n      & \dots  & v(r^T)
\end{bmatrix}
$$

where $\hat{r}$ has $n$ rows.

If we view $M_n$ as $v(M_n) \cong C^{n^2}$, then from the computation above,
we can guess that the matrix $M_r$ is:

$$
\hat{r} \otimes I_n
$$

The adjoint $w_r := m_r^{\dagger}$ is given by $W_r := M_r^{\dagger}$:

$$
w_r : a \mapsto W_cv(a)
$$

We can similarly compute $c_r := e_r^{\dagger}$:

$$
c_r : a \mapsto v(r^{-1})^Tv(a)
$$

We then verify the Frobenius laws.

In [3]:

def mltplr(R):
    m, n = R.dimensions()
    return mathat(R).tensor_product(I(m))

for N in range(1, 4):
    r = decl('r', N)
    R = v2m(r, N, N)
    M_R = mltplr(R)
    W_R = M_R.conjugate_transpose()

    assert((M_R.tensor_product(I(N^2))) * (I(N^2).tensor_product(W_R))\
           == W_R * M_R)
    assert((I(N^2).tensor_product(M_R)) * (W_R.tensor_product(I(N^2)))\
           == W_R * M_R)

print('Done')


Done


We then compute $W_rM_r$:

In [4]:

for N in range(1, 6):
    a = decl('a', N)
    r = decl('r', N)
    A = v2m(a, N, N)
    R = v2m(r, N, N)
    M_R = mltplr(R)
    W_R = mltplr(R).conjugate_transpose()

    assert(v2m(M_R * W_R * m2v(A), N, N) == norm(R) * A)

print('Done')


Done


We now try to compute $e_r^{\dagger}$:

In [5]:

N = 3
a = decl('a', N)
r = decl('r', N)
A = v2m(a, N, N)
R = v2m(r, N, N)
Rinv = matinv(R)

# show(Rinv.conjugate())

Rdag = matsimpl(matrix(m2v(Rinv.conjugate())))

# M = mltplr(R)
# W = M.conjugate_transpose()

show((R.conjugate().determinant() * Rdag * m2v(A))[0]\
       .simplify_rational()\
       .expand()\
       .simplify())


-a33*r12*r21 + a23*r13*r21 + a33*r11*r22 - a13*r13*r22 - a23*r11*r23 + a13*r12*r23 + a32*r12*r31 - a22*r13*r31 - a31*r22*r31 + a21*r23*r31 - a32*r11*r32 + a12*r13*r32 + a31*r21*r32 - a11*r23*r32 + a22*r11*r33 - a12*r12*r33 - a21*r21*r33 + a11*r22*r33